In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Define constants
A_s = 1.0 # amplitude of power spectrum 
Sigma_m_nu = 0.1 # eV
Omega_m_h2 = 0.14
f_nu = Sigma_m_nu / (93.14 * Omega_m_h2) 
n=1
m=4
k_eq = 0.015 # h/Mpc
k_nu = 0.05 # h/Mpc
Nk = 1e4 # number of modes per k bin


# Define k range
k_vals = np.arange(0.005, 0.201, 0.005)

In [5]:
# Precompute terms
def P_CDM(k):
   return k**n / (1 + (k / k_eq)**m)

def S(k, f_nu):
   return 1 - f_nu * (k**2 / (k**2 + k_nu**2))

def P(k, A_s, f_nu):
   return A_s * P_CDM(k) * S(k, f_nu)

# Derivatives
def dP_dAs(k):
   return P(k, A_s, f_nu) / A_s

def dP_dMnu(k):
   df_nu_dMnu = 1 / (93.14 * Omega_m_h2)
   dS_dfnu = -k**2 / (k**2 + k_nu**2)
   return A_s * P_CDM(k) * dS_dfnu * df_nu_dMnu

In [6]:
# Compute Fisher matrix components
F_AA = 0.0
F_Am = 0.0
F_mm = 0.0
for k in k_vals:
   Pk = P(k, A_s, f_nu)
   dP_A = dP_dAs(k)
   dP_m = dP_dMnu(k)
   factor = Nk / (2 * Pk**2)
   F_AA += factor * dP_A**2
   F_Am += factor * dP_A * dP_m
   F_mm += factor * dP_m**2
# Fisher matrix and covariance
F = np.array([
   [F_AA, F_Am],
   [F_Am, F_mm]
])
Cov = np.linalg.inv(F)
# Extract marginalized uncertainty on Sigma_m_nu
sigma_mnu = np.sqrt(Cov[1, 1])

In [7]:
# Print results
print("Fisher Matrix:\n", F)
print("\nCovariance Matrix:\n", Cov)
print("\n1-sigma uncertainty on sum neutrino mass: {:.4f}eV".format(sigma_mnu))

Fisher Matrix:
 [[200000.         -10498.9354489 ]
 [-10498.9354489     647.35238738]]

Covariance Matrix:
 [[3.36412224e-05 5.45602409e-04]
 [5.45602409e-04 1.03934806e-02]]

1-sigma uncertainty on sum neutrino mass: 0.1019eV
